# __Analysing Tweets to Verify the Presence of Toxic Speech With ToLD-Br__

In [1]:
import pandas as pd
from IPython.display import clear_output
from modules import preprocessing as pp
from simpletransformers.classification import ClassificationModel

/home/lucas/anaconda3/envs/twitter-cpi/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# data
tweets_df = pd.read_parquet('data/networks/complete_group_interacs.parquet')
tweets_df.shape

(1272586, 5)

In [3]:
tweets_df.sample(10)

,source,target,factions,interac_type,tweet_content
519662,Alessan79751750,roxmo,pro gov to neutral,quoted_rt,#BolsonaroPresidenteAte2026 🇧🇷Deus abençoe o B...
1265986,MaraCri40407638,camarotedacpi,against gov to neutral,reply,@camarotedacpi Vergonha senhores senadores #Bo...
1268620,MARCELO03904378,SenadoFederal,pro gov to neutral,reply,@SenadoFederal Vergonha pura dessa #CPIdoCirco!
136326,hbrumjunior,baia_canuto,pro gov to pro gov,quoted_rt,"Nossa , é isso q foi dito mesmo hoje no Senado..."
495500,raphaellazaro,JovemPanNews,pro gov to neutral,reply,@JovemPanNews O #RenanVagabundo não tem moral ...
234807,dani_jaming,AnarcoFino,neutral to neutral,reply,@AnarcoFino CPI do Genocídio vai virar púlpito...
122263,Jailson42394700,OmarAzizSenador,pro gov to neutral,reply,"@OmarAzizSenador Essa pesquisa é tão falsa, co..."
952573,Aldo08978663,jornalodia,pro gov to neutral,reply,@jornalodia #FechadoComBolsonaroAte2026 #que...
1119271,Diana_Rozza,Ticostacruz,against gov to against gov,mention,"""O silêncio também é uma posição política."" @T..."
1080960,Anderso89729239,o_antagonista,against gov to against gov,reply,@o_antagonista O Brasil vive a pior gestão de ...


In [4]:
tweets_df['factions'].value_counts()

pro gov to pro gov            420024
against gov to against gov    233985
against gov to neutral        210992
pro gov to neutral            150756
neutral to neutral            128118
pro gov to against gov         45385
against gov to pro gov         36404
neutral to against gov         36088
neutral to pro gov             10834
Name: factions, dtype: int64

In [8]:
model = ClassificationModel("distilbert", "toxic_bert_model", use_cuda=False)

In [9]:
n = 1
n_samples = 200
sample_dict = {}
predict_dict = {}
slices = ['pro gov to pro gov', 'against gov to against gov', 'against gov to neutral',
          'pro gov to neutral', 'neutral to neutral', 'pro gov to against gov',
          'against gov to pro gov']

In [10]:
%%time

for slice in slices:
    sample_dict[slice] = []
    predict_dict[slice] = []
    print(f'Starting prediction slice: {slice}')
    for x in range(n):
        print(f'Starting prediction number {x}...')
        samples = tweets_df[tweets_df['factions']==slice].sample(n_samples)
        sample_dict[slice].append(samples)
        predictions, outputs = model.predict(samples['tweet_content'].tolist())
        predict_dict[slice].append(predictions)

Starting prediction slice: pro gov to pro gov
Starting prediction number 0...


  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# pred_list = []
# count = 0

# for tweet in tweets_df['tweet_content'].tolist():
#     clear_output(wait=False)
#     print('Tweet number: ', count)
#     prediction, output = model.predict([tweet])
#     pred_list.append(prediction[0])
#     count+=1
# #print("Predictions:", predictions)
# #print("Outputs:", outputs)

In [ ]:
def pred_toxic(tweet: str):
    pred, out = model.predict([tweet])
    return pred[0]

In [ ]:
import scipy.stats as st
from math import ceil

# Cochran's formula
def get_sample_size(n_size, conf_level, margin_error, prop):
    z = st.norm.ppf(1-(1-conf_level)/2)
    n_0 = z**2*prop*(1-prop)/margin_error**2
    n = n_0/(1+(n_0-1)/n_size)
    n = ceil(n)
    return n

# Yamane's formula
def get_sample_size_2(n_size, e):
    n = n_size/(1+e**2)
    n = ceil(n)
    return n

In [ ]:
#n = 5
n = 5
#percent_val = 2
n_samples = 5000
#sample_dict = {}
#predict_dict = {}
#samples = []
sample_predicts = []
slices = ['pro gov to pro gov', 'against gov to against gov', 'against gov to neutral',
          'pro gov to neutral', 'neutral to neutral', 'pro gov to against gov',
          'against gov to pro gov']

#n_total_slice = {}
#for slice in slices:
#    n_total_slice[slice] = int(tweets_df['factions'].value_counts()[slice])

In [ ]:
get_sample_size(10000, 0.95, 0.05, 0.5)

In [ ]:
get_sample_size_2(400000, 0.05)

In [ ]:
#%%capture

#for slice in slices:
#    samples_slice = []
#    for x in range(n):
#        total_slice_size = tweets_df[tweets_df['factions']==slice].shape[0]
#        sample = tweets_df[tweets_df['factions']==slice].sample(x_perc_of(percent_val, total_slice_size))
#        sample['toxic_predict'] = sample['tweet_content'].map(lambda x: pred_toxic(pp.no_emoji_pipeline(x)))
#        samples_slice.append(sample)
#    samples.append(samples_slice)

In [ ]:
for slice in slices:
    #sample_dict[slice] = []
    #predict_dict[slice] = []
    print(f'Starting prediction slice: {slice}')
    for x in range(n):
        print(f'Starting prediction number {x}...')
        samples = tweets_df[tweets_df['factions']==slice].sample(n_samples)
        #n_sample = get_sample_size(n_total_slice[slice], 0.95, 0.05, 0.5)
        #samples = tweets_df[tweets_df['factions']==slice].sample(n_sample)
        samples_tt = [pp.no_emoji_pipeline(sample) for sample in samples['tweet_content'].tolist()]
        #sample_dict[slice].append(samples)
        predictions, outputs = model.predict(samples_tt)
        #predict_dict[slice].append(predictions)
        ##sample_predicts.extend(zip(samples['source'].tolist(), samples['target'].tolist(), samples['factions'].tolist(), samples['tweet_content'].tolist(),predictions))
        #for pred_tuples in zip(samples['source'].tolist(), samples['target'].tolist(), samples['factions'].tolist(), samples['tweet_content'].tolist(),predictions):
            #sample_predicts.append(pred_tuples)

In [ ]:
#del sample, samples_tt

sample2 = tweets_df.sample(10)
samples_tt2 = [pp.no_emoji_pipeline(sample) for sample in samples['tweet_content'].tolist()]

In [ ]:
predictions, outputs = model.predict(samples_tt)
predictions

In [ ]:
columns = ['source', 'target', 'factions', 'tweet_content', 'toxic_pred']
pd.DataFrame(columns=columns, data=sample_predicts)

In [ ]:
pd.Series(predict_dict['pro gov to pro gov'][0]).value_counts()

In [ ]:
pd.Series(predict_dict['against gov to against gov'][3]).value_counts()

In [ ]:
pd.Series(predict_dict['against gov to neutral'][2]).value_counts()

In [ ]:
pd.Series(predict_dict['pro gov to neutral'][2]).value_counts()

In [ ]:
pd.Series(predict_dict['pro gov to against gov'][1]).value_counts()

In [ ]:
pd.Series(predict_dict['against gov to pro gov'][1]).value_counts()

In [ ]:
pd.Series(predict_dict['neutral to neutral'][2]).value_counts()

In [ ]:
predict_dict['pro gov to pro gov'][0][:10]

In [ ]:
sample_dict['pro gov to pro gov'][0][:10]

## Faction Study

In [3]:
tweets_df[tweets_df['target']=='OmarAzizSenador'].sample()

,source,target,factions,interac_type,tweet_content
273880,carioca_lia,OmarAzizSenador,neutral to neutral,mention,Vocês vão deixar a Mayra Pinheiro fazer defesa...


In [4]:
tweets_df[tweets_df['target']=='randolfeap'].sample()

,source,target,factions,interac_type,tweet_content
723409,ErikakHilton,randolfeap,neutral to neutral,reply,@randolfeap Boa Senador! É escandalosa a atitu...


In [5]:
tweets_df[tweets_df['target']=='renancalheiros'].sample()

,source,target,factions,interac_type,tweet_content
1204960,julianamelles,renancalheiros,pro gov to neutral,mention,"@OmarAzizSenador , @renancalheiros , @randolfe..."


In [6]:
tweets_df[tweets_df['target']=='taoquei1'].sample()

,source,target,factions,interac_type,tweet_content
1136524,BenevidesThomas,taoquei1,pro gov to pro gov,reply,@taoquei1 Eu também #EuConfioNoPresidente Bras...


In [7]:
tweets_df[tweets_df['target']=='CarlaZambelli38'].sample()

,source,target,factions,interac_type,tweet_content
870414,luxintenebris7_,CarlaZambelli38,against gov to pro gov,reply,@CarlaZambelli38 @lipepaesleme @o_antagonista ...


In [8]:
tweets_df[tweets_df['target']=='MarcosRogerio'].sample()

,source,target,factions,interac_type,tweet_content
1020049,CentraldeCanc12,MarcosRogerio,against gov to neutral,mention,Esse senador @MarcosRogerio é um completo imbe...


In [9]:
tweets_df[tweets_df['target']=='kimpaim'].sample()

,source,target,factions,interac_type,tweet_content
686835,Jerrydevolta,kimpaim,pro gov to pro gov,reply,@kimpaim @LeilaDutra6 Relaxa...\n#RenanSabiaDe...


In [10]:
tweets_df[tweets_df['target']=='SenadoFederal'].sample()

,source,target,factions,interac_type,tweet_content
450115,Nanda23838332,SenadoFederal,pro gov to neutral,quoted_rt,Quero ver quebrar o sigilo do mandetta.\nEssa ...


In [11]:
tweets_df[tweets_df['target']=='Rconstantino'].sample()

,source,target,factions,interac_type,tweet_content
1120095,TeoMeow,Rconstantino,against gov to pro gov,reply,@Rconstantino #ForaBolsonaro Vcs se fuderam f...


In [12]:
tweets_df[tweets_df['target']=='BolsonaroSP'].sample()

,source,target,factions,interac_type,tweet_content
998451,CarlosR10950076,BolsonaroSP,against gov to pro gov,reply,@BolsonaroSP É a sua foto ao lado ANDA pensei ...


In [13]:
tweets_df[tweets_df['target']=='senadorhumberto'].sample()

,source,target,factions,interac_type,tweet_content
294526,vagnerpde,senadorhumberto,neutral to against gov,mention,@senadorhumberto Bom dia! Meu artigo sobre a f...


In [14]:
tweets_df[tweets_df['target']=='folha'].sample()

,source,target,factions,interac_type,tweet_content
883866,claudioinf,folha,against gov to neutral,reply,@folha Esse leitor deve ser um boi do bolsonar...


In [15]:
tweets_df[tweets_df['target']=='UOLNoticias'].sample()

,source,target,factions,interac_type,tweet_content
914971,nandoakr,UOLNoticias,pro gov to neutral,reply,"@UOLNoticias Mostrou que é possível fraudar, e..."


In [16]:
tweets_df[tweets_df['target']=='STF_oficial'].sample()

,source,target,factions,interac_type,tweet_content
986539,edsonbahiag4,STF_oficial,pro gov to neutral,mention,#TSEApagouAsProvas atentado a democracia é iss...


In [17]:
tweets_df[tweets_df['target']=='Sen_Alessandro'].sample()

,source,target,factions,interac_type,tweet_content
852875,n_maximoff,Sen_Alessandro,neutral to neutral,mention,A #CPIdaCovid tem que voltar depois da ordem d...


In [18]:
tweets_df[tweets_df['target']=='Metropoles'].sample()

,source,target,factions,interac_type,tweet_content
1233033,Pbmmesbr,Metropoles,against gov to neutral,quoted_rt,"""Expectativa : Siru vai comandar os protestos ..."


In [19]:
tweets_df[tweets_df['target']=='MidiaNINJA'].sample()

,source,target,factions,interac_type,tweet_content
1267695,CAntunes0,MidiaNINJA,neutral to neutral,quoted_rt,Que esse relatório e os crimes ecoem internaci...


In [20]:
tweets_df[tweets_df['target']=='Estadao'].sample()

,source,target,factions,interac_type,tweet_content
102081,NaldoValenca,Estadao,against gov to neutral,quoted_rt,O chefe da famiglia milíciana no poder central...


In [21]:
tweets_df[tweets_df['target']=='MarceloFreixo'].sample()

,source,target,factions,interac_type,tweet_content
107668,anselmo_sene,MarceloFreixo,against gov to neutral,reply,@MarceloFreixo Não conheço um país desenvolvid...


In [22]:
tweets_df[tweets_df['target']=='GuilhermeBoulos'].sample()

,source,target,factions,interac_type,tweet_content
1164893,EsquerdaPlease,GuilhermeBoulos,against gov to against gov,reply,@GuilhermeBoulos O que atrapalha é o excesso d...


In [23]:
tweets_df[tweets_df['target']=='o_antagonista'].sample()

,source,target,factions,interac_type,tweet_content
846272,SuzamaraSoares,o_antagonista,pro gov to against gov,reply,@o_antagonista Tá difícil né?\nDoeu muito ter ...


In [24]:
tweets_df[tweets_df['target']=='ContaratoSenado'].sample()

,source,target,factions,interac_type,tweet_content
923096,Critica_Nata,ContaratoSenado,against gov to against gov,quoted_rt,Deve com Guedes que o Bolsonás aprendeu que -4...


In [25]:
tweets_df[tweets_df['target']=='cirogomes'].sample()

,source,target,factions,interac_type,tweet_content
1014469,mdominguezbr,cirogomes,against gov to against gov,reply,"@cirogomes Traidor, mentiroso, genocida, incom..."


In [26]:
tweets_df[tweets_df['target']=='juliette'].sample()

,source,target,factions,interac_type,tweet_content
1108711,DrikaReis45,juliette,against gov to against gov,reply,@juliette É isso aí #ForaBolsonaro e #foralul...


In [27]:
tweets_df[tweets_df['target']=='brunogagliasso'].sample()

,source,target,factions,interac_type,tweet_content
1107916,renata_rela,brunogagliasso,against gov to against gov,reply,@brunogagliasso Bozo Flopou! O improvável é fl...


In [28]:
tweets_df[tweets_df['target']=='MBLivre'].sample()

,source,target,factions,interac_type,tweet_content
1130014,damattagab,MBLivre,against gov to against gov,reply,@MBLivre A notícia que recebi AGORA é MUITO BO...


In [29]:
tweets_df[tweets_df['target']=='jdoriajr'].sample()

,source,target,factions,interac_type,tweet_content
1034612,l_samar,jdoriajr,pro gov to against gov,reply,@jdoriajr Vamos agradecer ao Ministério da Saú...


In [30]:
tweets_df[tweets_df['target']=='veramagalhaes'].sample()

,source,target,factions,interac_type,tweet_content
802403,dyi11952124,veramagalhaes,against gov to neutral,reply,@veramagalhaes #CPIdaCovid @FFAABrasil além...


In [31]:
tweets_df[tweets_df['target']=='RubensNunesMBL'].sample()

,source,target,factions,interac_type,tweet_content
884275,Sonia_Pers,RubensNunesMBL,against gov to against gov,reply,@RubensNunesMBL Maior Estelionato Eleitoral da...


In [32]:
tweets_df[tweets_df['target']=='KimKataguiri'].sample()

,source,target,factions,interac_type,tweet_content
713276,ConcurseiroLima,KimKataguiri,against gov to against gov,reply,"@KimKataguiri Ele disse que se fosse pra ""faze..."


In [37]:
tweets_df[tweets_df['target']=='renancalheiros'].sample()

,source,target,factions,interac_type,tweet_content
150583,Leonardo16397,renancalheiros,pro gov to neutral,mention,@rpsenador \n@renancalheiros \n#CPIdoCirco \n#...


In [33]:
tweets_df[tweets_df['factions']=='against gov to against gov'].sample(20)

,source,target,factions,interac_type,tweet_content
368206,pnolasco2000,a_postagem,against gov to against gov,mention,CPI da Covid: Otto Alencar faz Dra. Nise Yamag...
1224613,angelacfgomes,ptbrasil,against gov to against gov,quoted_rt,Hoje é dia de luta na rua\n#Dia2ForaBolsonaro ...
1010832,eltoncpaulo,guimaraes13PT,against gov to against gov,reply,@guimaraes13PT São Paulo!!!\n#ForaBolsonaro
534306,jucampea_,vivireispsol,against gov to against gov,reply,@vivireispsol @juliette É isso #ForaBolsonaro
1151206,JulioK70,wvidal1906,against gov to against gov,reply,"@wvidal1906 @Nilsonhandebol Bora lá, companhei..."
1230379,guimaraes13PT,Larissafgaspar,against gov to against gov,mention,"Ao lado da Vereadora @Larissafgaspar, na luta ..."
1087149,GisaMaciel_,andrearoeirap,against gov to against gov,quoted_rt,Tanta terra pra boi e a população segue passan...
470703,MoshePhabio,JANI_VALVERD,against gov to against gov,reply,"@JANI_VALVERD Boçal, Bitolado, BolsoNarco. Cit..."
346193,ZheRhai,vivireispsol,against gov to against gov,reply,@vivireispsol Com distanciamento é mais emocio...
115920,felipepalha,henrychristian,against gov to against gov,reply,@henrychristian @VEJA O @desmentindobozo já pu...


In [34]:
tweets_df[tweets_df['factions']=='pro gov to neutral']['target'].value_counts()

OmarAzizSenador    21213
SenadoFederal      19172
MarcosRogerio       8611
renancalheiros      8171
SigaGazetaBR        7692
                   ...  
joseleonidas           1
pinkbloc_              1
seujosias1             1
true_news_now          1
marianasendoela        1
Name: target, Length: 2236, dtype: int64

In [35]:
tweets_df[tweets_df['factions']=='against gov to neutral']['target'].value_counts()

AAMachadoRP        12301
randolfeap         11253
UOLNoticias         9657
OmarAzizSenador     8223
MidiaNINJA          8044
                   ...  
ootimistajornal        1
LilianaRochaF          1
cruzeiro_mauro         1
printandonews          1
CarlosHortmann         1
Name: target, Length: 5116, dtype: int64

In [38]:
tweets_df[tweets_df['factions']=='pro gov to against gov']['target'].value_counts()[:20]

senadorhumberto    6176
revistaoeste       5839
o_antagonista      4790
jandira_feghali    1996
joaoamoedonovo     1385
DeputadoFederal    1220
jdoriajr           1068
MBLivre             836
KimKataguiri        827
erikakokay          791
pauloteixeira13     697
cirogomes           676
GuilhermeBoulos     657
joicehasselmann     502
janainalimasp       454
IvanValente         433
guimaraes13PT       426
samiabomfim         424
ptbrasil            358
taliriapetrone      338
Name: target, dtype: int64